In [12]:
import time
from pprint import pprint

In [13]:
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf

In [14]:
import sys

# Add the path to your module's directory
sys.path.append('/home/vladyslav_podrazhanskyi/projects/PERSONAL/python/learn_spark')

# Now you can import your 
from my_code.utils import ROOT

ROOT

PosixPath('/home/vladyslav_podrazhanskyi/projects/PERSONAL/python/learn_spark')

In [15]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

24/02/13 16:17:44 INFO SparkContext: Running Spark version 3.5.0
24/02/13 16:17:44 INFO SparkContext: OS info Linux, 6.5.0-17-generic, amd64
24/02/13 16:17:44 INFO SparkContext: Java version 1.8.0_392
24/02/13 16:17:44 INFO ResourceUtils: ==============================================================
24/02/13 16:17:44 INFO ResourceUtils: No custom resources configured for spark.driver.
24/02/13 16:17:44 INFO ResourceUtils: ==============================================================
24/02/13 16:17:44 INFO SparkContext: Submitted application: pyspark-shell
24/02/13 16:17:44 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
24/02/13 16:17:44 INFO ResourceProfile: Limiting resource is cpu
24/02/13 16:17:44 INFO ResourceProfileManager: Added

In [17]:
spark.sparkContext.setLogLevel("WARN")   # "INFO"

In [18]:
# spark catalog
catalog = spark.catalog
pprint(catalog.__sizeof__())
pprint(catalog.listDatabases())
pprint(catalog.listTables())

24
[Database(name='default', catalog='spark_catalog', description='default database', locationUri='file:/home/vladyslav_podrazhanskyi/projects/PERSONAL/python/learn_spark/my_code/my_practice/spark-warehouse')]
[]


UI: http://127.0.0.1:4040/jobs/

In [25]:
# spark reader
df = spark.read.format("json").load(f"{ROOT}/source_data/flight-data/json/")
df.show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 10 rows



In [26]:
df_renamed = df.withColumnRenamed("ORIGIN_COUNTRY_NAME", "renamed_ORIGIN_COUNTRY_NAME")

In [27]:
df_renamed.show()
print(df_renamed.count())  # 1502
df_renamed.printSchema()
print(df_renamed.schema)

+--------------------+---------------------------+-----+
|   DEST_COUNTRY_NAME|renamed_ORIGIN_COUNTRY_NAME|count|
+--------------------+---------------------------+-----+
|       United States|                    Romania|   15|
|       United States|                    Croatia|    1|
|       United States|                    Ireland|  344|
|               Egypt|              United States|   15|
|       United States|                      India|   62|
|       United States|                  Singapore|    1|
|       United States|                    Grenada|   62|
|          Costa Rica|              United States|  588|
|             Senegal|              United States|   40|
|             Moldova|              United States|    1|
|       United States|               Sint Maarten|  325|
|       United States|           Marshall Islands|   39|
|              Guyana|              United States|   64|
|               Malta|              United States|    1|
|            Anguilla|         

In [30]:
df_with_five = df.withColumn("five", sf.lit(5.0))
print("df_with_five")
df_with_five.show()
print("dif1_schema")
print(set(df_with_five.schema) - set(df.schema))
print("dif2_schema")
print(set(df.schema) - set(df_with_five.schema))

df_with_five
+--------------------+-------------------+-----+----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|five|
+--------------------+-------------------+-----+----+
|       United States|            Romania|   15| 5.0|
|       United States|            Croatia|    1| 5.0|
|       United States|            Ireland|  344| 5.0|
|               Egypt|      United States|   15| 5.0|
|       United States|              India|   62| 5.0|
|       United States|          Singapore|    1| 5.0|
|       United States|            Grenada|   62| 5.0|
|          Costa Rica|      United States|  588| 5.0|
|             Senegal|      United States|   40| 5.0|
|             Moldova|      United States|    1| 5.0|
|       United States|       Sint Maarten|  325| 5.0|
|       United States|   Marshall Islands|   39| 5.0|
|              Guyana|      United States|   64| 5.0|
|               Malta|      United States|    1| 5.0|
|            Anguilla|      United States|   41| 5.0|
|             B

In [36]:
df.select('count').show(3)
print(df.select('count').rdd)           # MapPartitionsRDD[27] at javaToPython at NativeMethodAccessorImpl.java:0
print(df.select('count').rdd.max())     # Row(count=370002)
print(df.select('count').rdd.max()[0])  # 370002

df.select(sf.max('count').alias('max')).show()
print(df.select(sf.max('count').alias('max')).collect()) # [Row(max=370002)]
print(df.select(sf.max('count').alias('max')).collect()[0]['max'])

+-----+
|count|
+-----+
|   15|
|    1|
|  344|
+-----+
only showing top 3 rows

MapPartitionsRDD[225] at javaToPython at NativeMethodAccessorImpl.java:0
Row(count=370002)
370002
+------+
|   max|
+------+
|370002|
+------+

[Row(max=370002)]
370002


In [37]:

df.select("ORIGIN_COUNTRY_NAME", "count").show(5)
df.selectExpr("ORIGIN_COUNTRY_NAME", "count").show(5)

df.select(
    "DEST_COUNTRY_NAME",
    sf.expr("DEST_COUNTRY_NAME"),
    sf.col("DEST_COUNTRY_NAME")
).show(5)

+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|            Romania|   15|
|            Croatia|    1|
|            Ireland|  344|
|      United States|   15|
|              India|   62|
+-------------------+-----+
only showing top 5 rows

+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|            Romania|   15|
|            Croatia|    1|
|            Ireland|  344|
|      United States|   15|
|              India|   62|
+-------------------+-----+
only showing top 5 rows

+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
|    United States|    United States|    United States|
|            Egypt|            Egypt|            Egypt|
|    United States|    United States|    United States

advantage of F.expr in comparison with col
df.select(F.expr("DEST_COUNTRY_NAME AS destination")).show(2)
df.select(F.col("DEST_COUNTRY_NAME").alias("destination")).show(2)

# selectExpr !!!
"""
Because select followed by a series of expr is such a common pattern, Spark has a shorthand
for doing this efficiently: selectExpr. This is probably the most convenient interface for
everyday use
This opens up the true power of Spark. We can treat selectExpr as a simple way to build up
complex expressions that create new DataFrames.
"""
df.selectExpr("DEST_COUNTRY_NAME as newColumnName", "DEST_COUNTRY_NAME").show(5)

df.selectExpr(
    "*",
    "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry") \
    .show(100)


# aggregations over the entire DataFrame

df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show(2)



print("sqlWay")

# temporary view for query with SQL
df.createOrReplaceTempView("dfTable")

sqlWayWithinCountry = spark.sql(
    """
    SELECT *,
    (DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry
    FROM dfTable
    """
).filter(sf.col('withinCountry')).show(12)

pprint(catalog.listTables()) # [Table(name='dftable', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

sqlWay = spark.sql("""
SELECT
DEST_COUNTRY_NAME, Sum(count) as sum
FROM dfTable
GROUP BY DEST_COUNTRY_NAME
ORDER by sum DESC""").show(10)

In [ ]:
time.sleep(600)
spar